In [3]:
# Example 07 - Using Functions

from dotenv import load_dotenv
import os
import requests
import json

### UTILITY FUNCTIONS

# This utility function is designed to submit the messages to AI's invoking OpenAI's URL direclty (using requests library)
# Please note we're not using openai python library here.
# Also please note that tools parameter will receive the function specification we created earlier
#GPT_MODEL= "gpt-3.5-turbo-0613"
GPT_MODEL= "gpt-4-0613"
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + os.environ.get("OPENAI_API_KEY"),
    }
    json_data = {"model": model, "messages": messages}
    if tools is not None:
        json_data.update({"tools": tools})
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def builds_message_with_instructions():
    messages = []
    instructions="""
        Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
        Now, if the question is not related to functions, than just answer normally.
        In case the user asks for some type of ranking, please respond in json format.
        EX: Question What are the greatest countries in the world for total area?
        {"ranking": [
            {"country_name": "country 1",
            "total_area": 1000
            },
            {"country_name": "country 2",
            "total_area": 500
            },
            {"country_name": "country 3",
            "total_area": 300
            }
        ]}
        The example is only for illustrating the output format, please substitute with proper data.
    """
    messages.append({"role": "system", "content": instructions})
    return messages

def builds_functions_definitions():
    ### Below, we're defining a function specification, where it expects one required argument (sku)
    tools = [
        {
            "type": "function",
            "function": {
                    "name": "get_inventory_quantity_for_sku",
                    "description": "Get the current Inventory for a given SKU",
                    "parameters": {
                        "type": "object",
                        "properties": {
                        "sku": {
                            "type": "string",
                            "description": "The SKU symbol"
                        }
                        },
                        "required": [
                        "sku"
                        ]
                    }
                    }
        }
    ]
    return tools


# Suppose you have an inventory and this function would return the quantity for a sku
# Here is hardcoded just for demonstration purposes
def get_inventory_quantity_for_sku(sku):
    # Dummy inventory data
    inventory = {"SKU12345": 10, "SKU67890": 5, "SKU11121": 0}
    return inventory.get(sku, "SKU not found in inventory")


# This function is doing the following:
#  1 -  from message received from AI, it's obtaining the function name and the parameters/arguments
#  2 -  invoking the function with the arguments
#  3 -  returning the results


def submit_user_prompt(prompt):

    messages = builds_message_with_instructions()
    # Setup functions to be used
    tools = builds_functions_definitions()
    messages.append({"role": "user", "content": prompt})
    response = chat_completion_request(messages, tools=tools).json()
    #print (json.dumps(response))
    if response['choices'][0]['message']['content'] is None and response['choices'][0]['message']['tool_calls'] is not None:
        # In case the response does not have any content, and it has tool_calls, it means chatgpt  is building
        # the function call with the parameters
        # To view it, uncomment the line below
        # print ("INTERMEDIATE RESPONSE", response)

        # Gets function to call
        function_to_call = (response['choices'][0]['message']['tool_calls'][0]['function']['name'])
        # Get Arguments/Parameters
        parameters = json.loads(response['choices'][0]['message']['tool_calls'][0]['function']['arguments'])
        function_pointer = eval(function_to_call)
        # Dynamically Invokes internal function (get_inventory_quantity_for_sku).. names and arguments must match !
        ret = function_pointer(**parameters)
    	# Appends a new message with role "function" with the actual content from the local function
        messages.append({"role": "function", "name": function_to_call, "content": str(ret)})
        # invokes GPT again, but now it will have the full context and variables to respond in natural language the SKU question
        response = chat_completion_request(
            messages, tools=tools
        ).json()

    return response['choices'][0]['message']['content']


### MAIN BODY (PROGRAM STARTS HERE)


response = submit_user_prompt(prompt="Can a man fly without any gadget ? ")
print ("1 ==> ", response)
response = submit_user_prompt(prompt="What is the quantity for SKU12345 ? ")
print ("2 ==> ",response)
response = submit_user_prompt(prompt="What are the greatest 4 cities in total area in the world ? ")
print ("3 ==> ",response)


1 ==>  No, a man cannot fly without any gadgets or supporting equipment. Despite being able to run, jump, and swim, humans do not have the physical capabilities to fly. This is due to several factors, such as the ratio of muscle mass to body size, lack of wings, and body structure. The dream of human flight can, however, be achieved with the aid of technology like airplanes, helicopters, jet packs, paragliders, and skydiving equipment.
2 ==>  The quantity for SKU12345 is 10.
3 ==>  {"ranking": [
    {"city_name": "Hulunbuir",
    "total_area": 263953
    },
    {"city_name": "Altamira",
    "total_area": 159533
    },
    {"city_name": "Baotou",
    "total_area": 27709
    },
    {"city_name": "New York City",
    "total_area": 783.8
    }
]}
